# Basic

In [74]:
import pandas as pd
import time
from tqdm import tqdm
import requests
import json

raw_path = "./data/raw/"
temp_path = "./data/temp/"
data_path = "./data/"

In [20]:
def reduce_df(df):
    df = df[(df["Latitude"]>37.454) & (df["Latitude"]<37.536) & (df["Longitude"]>127.018) & (df["Longitude"]<127.1266)]
    df = df.reset_index(drop=True)
    return df


def whether_gangnam(latitude, longitude):    
    url = "https://apis.openapi.sk.com/tmap/geo/reversegeocoding?version=1"

    payload = {
        "appKey": json.load(open("./data/secrets.json", "r"))["tmap"]["appkey"],
        "lon": longitude,
        "lat": latitude,
    }

    res = requests.get(url, params=payload)

    try:
        gu = json.loads(res.text)["addressInfo"]["gu_gun"]
    except:
        return "ERROR"

    if gu == "강남구":
        return True
    else:
        return False

# 버스정류장

In [21]:
bus = pd.read_excel(raw_path + "서울시 버스정류소 좌표데이터(2022.11.30).xlsx")
bus = bus[["정류소명", "Y좌표", "X좌표"]]
bus = bus.rename({"정류소명": "Name", "Y좌표": "Latitude", "X좌표": "Longitude"}, axis=1)
bus = reduce_df(bus)
print(len(bus))
bus.head()

1126


,Name,Latitude,Longitude
0,성수동차고지,37.535793,127.055682
1,자양미소약국.자양골목시장,37.532750,127.077319
2,국민은행자양지점앞,37.531573,127.081413
3,자양아파트앞,37.532192,127.084608
4,자양동잠실대교북단,37.532955,127.084806


In [22]:
for i in tqdm(range(len(bus))):
    bus.loc[i, "gangnam"] = whether_gangnam(bus.loc[i, "Latitude"], bus.loc[i, "Longitude"])

100%|██████████████████████████████████████████████████████████████████████████████| 1126/1126 [03:15<00:00,  5.77it/s]


In [23]:
bus = bus[bus["gangnam"]==True]
bus = bus.drop(["gangnam"], axis=1).reset_index(drop=True).reset_index()

print(len(bus))
print(bus.info())
bus.head()

551
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551 entries, 0 to 550
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   index      551 non-null    int64  
 1   Name       551 non-null    object 
 2   Latitude   551 non-null    float64
 3   Longitude  551 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 17.3+ KB
None


,index,Name,Latitude,Longitude
0,0,뱅뱅사거리,37.486653,127.033097
1,1,래미안아파트.파이낸셜뉴스,37.491087,127.030987
2,2,신분당선강남역,37.494993,127.029112
3,3,지하철2호선강남역,37.500785,127.026370
4,4,논현역,37.506367,127.023728


In [26]:
bus.to_csv(temp_path + "bus.csv", index=False, encoding="utf-8")

# 택시승강장

In [27]:
taxi = pd.read_csv(raw_path + "서울특별시_택시승차대 현황_20210731.csv", encoding="cp949")
taxi = taxi[(taxi["전원투입"]=="OK") & (taxi["행정구"]=="강남구")][["위치명", "좌표(Poi_Y)", "좌표(Poi_X)"]]
taxi = taxi.rename({"위치명": "Name", "좌표(Poi_Y)": "Latitude", "좌표(Poi_X)": "Longitude"}, axis=1).reset_index(drop=True).reset_index()
taxi.loc[:, "Longitude"] = taxi["Longitude"].apply(lambda x: float(x.replace("/", ".")))

print(len(taxi))
print(taxi.info())
taxi.head()

14
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   index      14 non-null     int64  
 1   Name       14 non-null     object 
 2   Latitude   14 non-null     float64
 3   Longitude  14 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 576.0+ bytes
None


C:\Users\ParkMinjeong\AppData\Local\Temp\ipykernel_12528\3940924088.py:4: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  taxi.loc[:, "Longitude"] = taxi["Longitude"].apply(lambda x: float(x.replace("/", ".")))


,index,Name,Latitude,Longitude
0,0,어반테일러 앞,37.508280,127.022290
1,1,가로수길 영동빌딩 (올리브영 가로수길 중앙점 앞),37.518136,127.022959
2,2,임페리얼팰리스호텔 앞,37.513700,127.035520
3,3,한국건설기술인협회별관 좌측(상아아파트 건너),37.516880,127.051360
4,4,삼원가든 앞,37.525000,127.033110


In [28]:
taxi.to_csv(temp_path + "taxi.csv", index=False, encoding="utf-8")

# subway

In [29]:
# 1 ~ 8호선
subway_1to8 = pd.read_csv(raw_path + "서울교통공사_1_8호선 역사 좌표(위경도) 정보_20211231.csv", encoding="cp949")
subway_1to8["Name"] = subway_1to8.apply(lambda x: f"{x['호선']}호선 {x['역명']}역", axis=1)
subway_1to8 = subway_1to8[["Name", "위도", "경도"]]
subway_1to8 = subway_1to8.rename({"위도": "Latitude", "경도": "Longitude"}, axis=1)

print(len(subway_1to8))
print(subway_1to8.info())
subway_1to8.head()

276
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       276 non-null    object 
 1   Latitude   276 non-null    float64
 2   Longitude  276 non-null    float64
dtypes: float64(2), object(1)
memory usage: 6.6+ KB
None


,Name,Latitude,Longitude
0,1호선 서울역,37.553150,126.972533
1,1호선 시청역,37.563590,126.975407
2,1호선 종각역,37.570203,126.983116
3,1호선 종로3가역,37.570429,126.992095
4,1호선 종로5가역,37.570971,127.001900


In [30]:
# 9호선
subway_9 = pd.read_csv(path + "국가철도공단_수도권9호선_역위치_20221117.csv").dropna()
subway_9["Name"] = subway_9.apply(lambda x: f"{x['선명']} {x['역명']}역", axis=1)
subway_9 = subway_9[["Name", "위도", "경도"]]
subway_9 = subway_9.rename({"위도": "Latitude", "경도": "Longitude"}, axis=1)

print(len(subway_9))
print(subway_9.info())
subway_9.head()

29
<class 'pandas.core.frame.DataFrame'>
Int64Index: 29 entries, 0 to 28
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       29 non-null     object 
 1   Latitude   29 non-null     float64
 2   Longitude  29 non-null     float64
dtypes: float64(2), object(1)
memory usage: 928.0+ bytes
None


,Name,Latitude,Longitude
0,9호선 개화역,37.578608,126.798153
1,9호선 김포공항역,37.562434,126.801058
2,9호선 공항시장역,37.563726,126.810678
3,9호선 신방화역,37.567532,126.816601
4,9호선 마곡나루역,37.567336,126.829497


In [32]:
subway = pd.concat([subway_1to8, subway_9], ignore_index=True)
subway = reduce_df(subway)
print(len(subway))
print(subway.info())
subway.head()

40
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       40 non-null     object 
 1   Latitude   40 non-null     float64
 2   Longitude  40 non-null     float64
dtypes: float64(2), object(1)
memory usage: 1.1+ KB
None


,Name,Latitude,Longitude
0,2호선 강변역,37.535161,127.094684
1,2호선 잠실나루역,37.520688,127.103836
2,2호선 잠실역,37.513305,127.100129
3,2호선 잠실새내역,37.520731,127.103738
4,2호선 종합운동장역,37.511008,127.073641


In [33]:
for i in tqdm(range(len(subway))):
    subway.loc[i, "gangnam"] = whether_gangnam(subway.loc[i, "Latitude"], subway.loc[i, "Longitude"])

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:06<00:00,  5.87it/s]


In [34]:
subway = subway[subway["gangnam"]==True]
subway = subway.drop(["gangnam"], axis=1).reset_index(drop=True).reset_index()

print(len(subway))
print(subway.info())
subway.head()

22
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   index      22 non-null     int64  
 1   Name       22 non-null     object 
 2   Latitude   22 non-null     float64
 3   Longitude  22 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 832.0+ bytes
None


,index,Name,Latitude,Longitude
0,0,2호선 삼성역,37.508827,127.063203
1,1,2호선 선릉역,37.504257,127.048174
2,2,2호선 역삼역,37.500658,127.036430
3,3,3호선 압구정역,37.526169,127.028502
4,4,3호선 신사역,37.516438,127.020247


In [35]:
subway.to_csv(temp_path + "subway.csv", index=False, encoding="utf-8")

# 소방용수

In [61]:
fire = pd.read_csv(raw_path + "서울시 소방재난본부_소방용수시설_20220831.csv", encoding="cp949")
fire = fire[(fire["시군구명"]=="강남구") & (fire["시설유형코드"]==1) & (fire["사용가능여부"]=="Y")][["상세위치", "위도", "경도"]]
fire = fire.rename({"상세위치": "Name", "위도": "Latitude", "경도": "Longitude"}, axis=1).reset_index(drop=True).reset_index()

print(len(fire))
print(fire.info())
fire.head()

966
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966 entries, 0 to 965
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   index      966 non-null    int64  
 1   Name       679 non-null    object 
 2   Latitude   966 non-null    float64
 3   Longitude  966 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 30.3+ KB
None


,index,Name,Latitude,Longitude
0,0,서광아파트102동,37.517144,127.042529
1,1,NaN,37.499261,127.047582
2,2,서울비젼 사옥,37.511763,127.035584
3,3,미진프라자,37.497998,127.028693
4,4,대치MS빌딩,37.500777,127.052224


In [62]:
fire.to_csv(temp_path + "fire.csv", index=False, encoding="utf-8")

# 어린이 보호구역

In [66]:
children = pd.read_csv(raw_path + "어린이보호구역 위치도.csv", encoding="cp949")
children = children[children["구"]=="강남구"][["시설명", "y좌표", "x좌표"]]
children = children.rename({"시설명": "Name", "y좌표": "Latitude", "x좌표": "Longitude"}, axis=1).reset_index(drop=True).reset_index()

print(len(children))
print(children.info())
children.head()

32
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   index      32 non-null     int64  
 1   Name       32 non-null     object 
 2   Latitude   32 non-null     float64
 3   Longitude  32 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 1.1+ KB
None


,index,Name,Latitude,Longitude
0,0,성모어린이집,37.510980,127.026898
1,1,논현초등학교,37.508170,127.026032
2,2,세리유치원,37.520723,127.025976
3,3,신사어린이집,37.522080,127.040066
4,4,청담어린이집,37.522080,127.040023


In [67]:
children.to_csv(temp_path + "children.csv", index=False, encoding="utf-8")

# concat

In [68]:
data_list = ["bus", "taxi", "subway", "fire", "children"]

In [72]:
cautionzone = pd.DataFrame(columns=["index", "Name", "Latitude", "Longitude", "type"])

for data in data_list:
    data_df = pd.read_csv(f"{temp_path}{data}.csv")
    data_df["type"] = data
    cautionzone = pd.concat([cautionzone, data_df], ignore_index=True)

print(len(cautionzone))
print(cautionzone.info())
cautionzone.head()

1585
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1585 entries, 0 to 1584
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   index      1585 non-null   object 
 1   Name       1298 non-null   object 
 2   Latitude   1585 non-null   float64
 3   Longitude  1585 non-null   float64
 4   type       1585 non-null   object 
dtypes: float64(2), object(3)
memory usage: 62.0+ KB
None


,index,Name,Latitude,Longitude,type
0,0,뱅뱅사거리,37.486653,127.033097,bus
1,1,래미안아파트.파이낸셜뉴스,37.491087,127.030987,bus
2,2,신분당선강남역,37.494993,127.029112,bus
3,3,지하철2호선강남역,37.500785,127.026370,bus
4,4,논현역,37.506367,127.023728,bus


In [73]:
cautionzone.to_csv(data_path + "cautionzone.csv", index=False, encoding="utf-8")

In [76]:
cautionzone = pd.read_csv(data_path + "cautionzone.csv")

In [140]:
cautionzone.head()

,index,Name,Latitude,Longitude,type
0,0,뱅뱅사거리,37.486653,127.033097,bus
1,1,래미안아파트.파이낸셜뉴스,37.491087,127.030987,bus
2,2,신분당선강남역,37.494993,127.029112,bus
3,3,지하철2호선강남역,37.500785,127.026370,bus
4,4,논현역,37.506367,127.023728,bus


In [144]:
for i in tqdm(range(len(cautionzone))):
    cautionzone.loc[i, "dongCode"] = get_dong(cautionzone.loc[i, "Latitude"], cautionzone.loc[i, "Longitude"])

100%|██████████████████████████████████████████████████████████████████████████████| 1585/1585 [03:41<00:00,  7.15it/s]


In [145]:
cautionzone["dongCode"].unique()

array(['118', '101', '108', '105', '107', '110', '115', '104', '106',
       '112', '103', '114', '113', '111', 'Invalid Code'], dtype=object)

In [147]:
cautionzone.to_csv(data_path + "cautionzone.csv", index=False, encoding="utf-8")

In [143]:
def get_dong(latitude, longitude):
    url = "https://apis.openapi.sk.com/tmap/geo/reversegeocoding?version=1"

    payload = {
        "appKey": "l7xx0ffb87c22bdb45ae8facaeeb7958ad36",
        "lon": longitude,
        "lat": latitude,
    }

    res = requests.get(url, params=payload)

    try:
        dongCode = json.loads(res.text)["addressInfo"]["legalDongCode"][5:8]
    except:
        return "ERROR"

    if dongCode in dong["dongCode"].unique():
        return dongCode
    else:
        return "Invalid Code"

In [163]:
cautionzone = pd.read_csv(data_path+"cautionzone.csv").iloc[:, 1:]
cautionzone = cautionzone.rename({"index":"id", "Name": "name", "Latitude": "latitude", "Longitude": "longitude"}, axis=1)
cautionzone.to_csv(data_path+"cautionzone.csv", index=False)

In [161]:
cautionzone

,id,name,latitude,longitude,type,dongCode
0,0,뱅뱅사거리,37.486653,127.033097,bus,118
1,1,래미안아파트.파이낸셜뉴스,37.491087,127.030987,bus,101
2,2,신분당선강남역,37.494993,127.029112,bus,101
3,3,지하철2호선강남역,37.500785,127.026370,bus,101
4,4,논현역,37.506367,127.023728,bus,108
...,...,...,...,...,...,...
1580,27,보미나어린이집,37.511241,127.028969,children,108
1581,28,강남구청어린이집,37.517528,127.047470,children,105
1582,29,연화어린이집,37.517216,127.037152,children,108
1583,30,봉은초등학교,37.520135,127.060981,children,105


In [164]:
parkingzone = pd.read_csv(data_path+"parkingzone.csv").iloc[:, 1:]
parkingzone = parkingzone.rename({"index":"id", "Name": "name", "Latitude": "latitude", "Longitude": "longitude"}, axis=1)
parkingzone.to_csv(data_path+"parkingzone.csv", index=False)

In [156]:
parkingzone

,id,name,latitude,longitude,type,dongCode
0,0,서울 강남구 청담동 41-2,37.517641,127.041673,kickboard,104
1,1,서울 강남구 삼성동 145,37.512883,127.053852,kickboard,105
2,2,서울 강남구 역삼동 717,37.500605,127.038096,kickboard,101
3,3,서울 강남구 삼성동 52,37.518540,127.050709,kickboard,105
4,4,서울 강남구 논현동 129-1,37.513568,127.030541,kickboard,108
5,5,서울 강남구 신사동 501-2,37.516780,127.020108,kickboard,107
